In this second experiment, we compare RangeGrad2 to a number of existing methods. Here, we use the VGG19 network trained on the ImageNet dataset. We compare our results to the simple gradient (baseline), SmoothGrad and GradCam.

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from random import choices
from minmax.lmm_models import vgg, resnet, densenet
from torchvision.models import vgg as vggn
from time import time
from minmax.ImageNetClasses import classes

import warnings
warnings.filterwarnings('ignore')

data_transforms = {
    'train': T.Compose([
        T.Resize((224,224)),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'inverse': T.Compose([
        T.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
        T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    ]),
}

testset = torchvision.datasets.ImageNet(
    root='~/datasets/ImageNet',
    split="val",
    transform=data_transforms['val']
)

#samples = [8922,13899,16096,4753,9301,37782,35062,25899,23031,44597] + choices(range(len(testset)), k=10) 
#samples = [18835, 14470, 33868, 25621, 33312, 40082, 46358]# + choices(range(len(testset)), k=50)
samples = choices(range(len(testset)), k=10)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=1,
    num_workers=2,
    sampler=samples
    #shuffle=True
)

netn = vggn.vgg19(pretrained=True)
netn.eval()

net = vgg.vgg19(pretrained=True)
net.eval()

print("Loading Complete")

In [ ]:
import matplotlib.pyplot as plt
import cv2
from minmax.mm_linearize import mmTensor, explainV1, explainV2, explainGradient
import scipy.ndimage as ndimage
import os
from minmax.ImageNetClasses import classes
from pytorch_grad_cam import GradCAM
import numpy as np
from datetime import datetime
from captum.attr import (
    Occlusion,
    NoiseTunnel,
    DeepLiftShap,
    Saliency,
    GuidedGradCam
)

OUTPUT_FOLDER = "Experiment_2"
try:
    os.mkdir(OUTPUT_FOLDER)
    print("Output Folder Created")
except FileExistsError:
    print("Output Folder Exists")

for i, (data, target) in enumerate(testloader):
    # Plot all versions of the explanation
    fig, axs = plt.subplots(1,7)
    fig.set_size_inches(17,10)
    
    info = {
        'out': OUTPUT_FOLDER,
        'snum': samples[i],
        'class': classes[target.item()]
    }
    
    # Guided GradCam
    info['method'] = 'Guided-GradCam'
    expl = GuidedGradCam(netn, netn.features[-1])
    explGuidedGradCam = expl.attribute(data, target=target)
    explGuidedGradCam = explGuidedGradCam.detach().numpy()
    explGuidedGradCam = explGuidedGradCam.sum(axis=0)
    explGuidedGradCam = np.linalg.norm(explGuidedGradCam, axis=0)
    
    axs[0].axis("off")
    axs[0].imshow(explGuidedGradCam, cmap=plt.cm.Reds)
    axs[0].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explGuidedGradCam, cmap=plt.cm.Reds)

    # SmoothGrad
    info['method'] = 'SmoothGrad'
    timeSmoothGrad = datetime.now()
    saliency = Saliency(netn)
    nt = NoiseTunnel(saliency)
    explSmoothGrad = nt.attribute(data, target=target, nt_type='smoothgrad', nt_samples=15)
    explSmoothGrad = explSmoothGrad.detach().numpy().sum(axis=0)
    explSmoothGrad = np.linalg.norm(explSmoothGrad, axis=0)
    
    axs[1].axis("off")
    axs[1].imshow(explSmoothGrad, cmap=plt.cm.Reds)
    axs[1].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explSmoothGrad, cmap=plt.cm.Reds)
    
    # DeepLiftShap
    info['method'] = 'DeepLiftShap'
    timeShapley = datetime.now()
    dist = torch.cat([data * 0, data * 1])
    expl = DeepLiftShap(netn)
    explShapley = expl.attribute(data, target=target, baselines=dist)
    explShapley = explShapley.detach().numpy().sum(axis=0)
    explShapley = np.linalg.norm(explShapley, axis=0)
    
    axs[2].axis("off")
    axs[2].imshow(explShapley, cmap=plt.cm.Reds)
    axs[2].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explShapley, cmap=plt.cm.Reds)

    # Occlusion
    info['method'] = 'Occlusion'
    expl = Occlusion(netn)
    explOcclusion = expl.attribute(data, target=target, strides=(3,25,25), sliding_window_shapes=(3,50,50))
    explOcclusion = explOcclusion.detach().numpy().sum(axis=0).sum(axis=0)
    if not target: explOcclusion = -explOcclusion
    explOcclusion -= explOcclusion.min()
    explOcclusion /= explOcclusion.max()
    
    axs[3].axis("off")
    axs[3].imshow(explOcclusion, cmap=plt.cm.Reds)
    axs[3].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explOcclusion, cmap=plt.cm.Reds)

    # Gradient
    info['method'] = 'Gradient'
    explGradient, _ = explainGradient(net, data)
    
    axs[4].axis("off")
    axs[4].imshow(explGradient, cmap=plt.cm.Reds)
    axs[4].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explGradient, cmap=plt.cm.Reds)

    # RangeGrad
    info['method'] = 'RangeGrad'
    explRangeGrad, _ = explainV2(net, data, scale=1/100, width=1/1000)
    axs[5].axis("off")
    axs[5].imshow(explRangeGrad, cmap=plt.cm.Reds)
    axs[5].set_title(info['method'])
    plt.imsave("{out}/{snum}_{method}.png".format(**info), explRangeGrad, cmap=plt.cm.Reds)

    # Image
    image = data_transforms['inverse'](data)[0].movedim(0,2).numpy()
    axs[6].axis("off")
    axs[6].imshow(image)
    axs[6].set_title("Original {} - {}".format(samples[i], classes[target.item()]))
    plt.imsave("{out}/{snum}_original.png".format(**info), image)
    plt.savefig("{out}/{snum}.png".format(**info))
    plt.show()